# U.S. Medical Insurance Costs

Goal: The goal of this project is to optimize the insurance cost in my future. This project will help me decide where to live if I want to have children and what my bmi should be (regarding bmi I will try to inspect if lower means alwas better). 

Data: For this project I am assuming that insurance is provided by one company. Also In the main part I won't focus on sex and smoking since I want change my sex and I will never smoke. So I will filter them out

## Preparing Data
Firstly I have imported pandas library and insurance.csv

In [2]:
import csv
import pandas as pd

starting_df = pd.read_csv("insurance.csv")
test = 1111



Now I want to check if there are any rows with Null or other "bad" values.

In [3]:

def check_for_nulls (df):
    df["nulls"] = df.apply(lambda row: row.isnull().any(), axis=1)
    return df
test_null = check_for_nulls(starting_df)

print(test_null["nulls"].unique())



[False]


In [4]:
starting_df.drop(columns=['nulls'], inplace=True)
print(starting_df.columns)

Index(['age', 'sex', 'bmi', 'children', 'smoker', 'region', 'charges'], dtype='object')


## Minimizing My Insurance Cost: 

In [5]:
#Jakub's Code

## Evaluating Cost-BMI relationship:
### Steps Brainstorm:
 - What is the average cost of insurance. the ave bmi?
 

In [ ]:
#Kaileb's Code
starting_df.head()


,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [ ]:
#Assess Data set as a whole

#Cost to BMI Frequency cart
bmi_cost_freq = starting_df.bmi.apply(lambda bmi_float : round(bmi_float)).reset_index() 
print(bmi_cost_freq.head(10))

bmi_cost_2 = starting_df.groupby(starting_df.bmi.apply(lambda bmi_float : round(bmi_float))).charges.count().reset_index().rename(columns={'charges': 'bmi-freq'})
print(bmi_cost_2.head(10))
bmi_cost_3 = starting_df.groupby(starting_df.bmi.apply(lambda bmi_float : round(bmi_float))).charges.mean().reset_index()
print(bmi_cost_3.head(10))

   index  bmi
0      0   28
1      1   34
2      2   33
3      3   23
4      4   29
5      5   26
6      6   33
7      7   28
8      8   30
9      9   26
   bmi  bmi-freq
0   16         1
1   17         9
2   18        11
3   19         8
4   20        24
5   21        23
6   22        35
7   23        50
8   24        57
9   25        62
   bmi       charges
0   16   1694.796400
1   17   6076.167378
2   18  11402.702809
3   19  10382.876080
4   20   8756.253285
5   21   7006.185911
6   22  10432.877525
7   23  10104.465317
8   24  11675.051279
9   25  12203.856403
